# 🤖 Chatbot d'Orientation Scolaire avec NLTK et Transformers
Ce notebook montre comment créer un chatbot qui répond aux questions d'orientation scolaire en utilisant **NLTK, TF-IDF et un modèle Transformers**.

Ce code importe et configure plusieurs bibliothèques essentielles pour le traitement du langage naturel (NLP). Voici une explication détaillée de chaque partie :

## 📌 1. Importation des bibliothèques

Nous aurons besoin des bibliothèques suivantes :

*   pandas : utilisé pour manipuler des données sous forme de tableaux(DataFrame).
*  nltk (Natural Language Toolkit) : une bibliothèque puissante pour le NLP.
*   word_tokenize : permet de diviser un texte en mots individuels (tokenisation).
*   stopwords : contient une liste de mots courants à exclure (comme "le","et", "de", etc.).
*   TfidfVectorizer (de sklearn.feature_extraction.text) : convertit un texte en représentation numérique en utilisant la pondération TF-IDF.
*   cosine_similarity (de sklearn.metrics.pairwise) : mesure la similarité entre deux textes sur la base de leurs représentations numériques.
*   pipeline (de transformers) : permet d'utiliser facilement des modèles de deep learning pour le NLP, comme ceux de Hugging Face.


In [37]:

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# Télécharger les ressources NLTK nécessaires
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rayenhaddad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rayenhaddad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 📌 2. Chargement du dataset
Nous allons utiliser pandas pour charger un dataset contenant du texte. Ce dataset peut être sous forme de CSV, JSON ou TXT, selon le type de tâche que nous souhaitons effectuer.

Pour une première visualisation, pensez à telecharger
[dataset](https://https://drive.google.com/file/d/1DZPS5Vtq0Cs0qZCUXJXv8FE80GDaSN2-/view?usp=drive_link) proposer.

Vous pourrez le modifier suite à votre enrichissement de données.

Pensez à visualiser le dataset avant de lancer votre modèle dessus.

In [38]:

# Charger le dataset
import requests


df = pd.read_csv("../../data/dataset_orientation_scolaire.csv")


# df = pd.read_csv("/content/questions_orientation_scolaire.csv") # à modifier avec votre dataset suite à l'enrichissement des données.

# Afficher les premières lignes
df.head()


,Question,Réponse
0,Quels sont les métiers du numérique ?,Les métiers du numérique incluent développeur ...
1,Quelles compétences sont requises pour devenir...,"Maîtrise des langages HTML, CSS, JavaScript, a..."
2,Quels métiers peut-on exercer avec un diplôme ...,"Chargé de communication, chef de produit, resp..."
3,Quels sont les métiers de la finance ?,"Analyste financier, contrôleur de gestion, com..."
4,Comment devenir architecte ?,"Faire un Bac S ou STI2D, puis intégrer une éco..."


## 📌 3. Nettoyage et Pré-traitement des questions



## 📝 Nettoyage de Texte avec NLTK

On va utiliser la bibliothèque **NLTK** pour nettoyer du texte en français. Il effectue les opérations suivantes :
- Télécharge et charge la liste des **stopwords** (mots vides) en français.
- Tokenise (*découpe*) le texte en mots individuels.
- Supprime les **stopwords** et les caractères non alphanumériques.
- Implémente une **gestion des erreurs** pour éviter les problèmes liés aux ressources NLTK.

Ce prétraitement est essentiel pour de nombreuses tâches de Large Language Model ou même NLP

In [39]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Télécharger les ressources NLTK nécessaires
# TO DO
# ....

# Charger les stopwords français
stop_words = set(stopwords.words('french'))  # TO DO: Charger les stopwords de la langue française

# Fonction de nettoyage avec gestion d'erreur
def nettoyer_texte(texte):
    try:
        tokens = word_tokenize(texte.lower())  # Tokenisation avec NLTK
    except LookupError:
        tokens = texte.lower().split()  # Mode de secours en cas d'erreur

    tokens = [mot for mot in tokens if mot.isalnum() and mot not in stop_words]  # Suppression des stopwords et caractères spéciaux
    return " ".join(tokens)

# Test du nettoyage
texte_test = "Quels sont les débouchés après le Bac ?"
print(nettoyer_texte(texte_test))

quels débouchés après bac


Que constatez vous ?

In [ ]:
# On remarque que les stopwords ne sont plus présents (tel que sont, le et ?). On comprend encore le sens du texte mais on l'a simplifié

Appliquons le à l'ensemble de notre dataset.

In [40]:
df["Question_clean"] = df["Question"].apply(nettoyer_texte)

## 4. 🎯 Objectif : Trouver une Réponse à une Question avec TF-IDF

Dans cette activité, nous allons utiliser une **approche classique de NLP (Traitement Automatique du Langage Naturel)** pour associer une question posée par un utilisateur à la réponse la plus pertinente dans une base de données.

### 🛠 Ce que nous allons faire :
1. **Vectoriser les questions** avec la méthode **TF-IDF (Term Frequency - Inverse Document Frequency)** pour les transformer en représentation numérique.
2. **Comparer une nouvelle question** avec celles existantes en calculant la **similarité cosinus**.
3. **Identifier la réponse la plus pertinente** en sélectionnant la question la plus proche de celle posée.

💡 **Pourquoi cette méthode ?**  
Lorsqu'un modèle n'est pas encore entraîné pour comprendre directement les questions et fournir des réponses, **TF-IDF** permet de trouver une correspondance basée sur la proximité des mots et leur importance dans le texte.

✅ **À vous de jouer !** Testez le code en posant différentes questions et observez la réponse retournée. Vous pouvez également modifier la base de données pour voir comment le modèle s’adapte.


In [41]:
# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["Question_clean"])

def trouver_reponse(question):
    question_clean = nettoyer_texte(question)
    question_tfidf = vectorizer.transform([question_clean])
    similarites = cosine_similarity(question_tfidf, tfidf_matrix)
    index_meilleure = similarites.argmax()
    return df.iloc[index_meilleure]["Réponse"]


## 5. 📝 Test du Chatbot avec une Question Utilisateur

Ce code permet de tester notre **chatbot basé sur TF-IDF** en lui posant une question.  

### 🔹 Fonctionnement :
1. **Une question utilisateur est définie**, par exemple :  
   *"Comment intégrer une école de médecine ?"*
2. **La fonction `trouver_reponse()` est appelée** pour analyser la question et rechercher la réponse la plus pertinente dans notre base de données.
3. **Le chatbot affiche la réponse trouvée**, basée sur la similarité avec les questions existantes.

💡 **Essayez avec différentes questions et observez les résultats !**  
Vous pouvez également modifier la base de données pour voir comment cela influence les réponses du chatbot.


In [42]:
# Test du chatbot
question_utilisateur = "Comment devenir architect?"
print("Edu Pilot:", trouver_reponse(question_utilisateur))

Edu Pilot: Faire un Bac S ou STI2D, puis intégrer une école d'architecture reconnue par l'État. Compétences : créativité, maîtrise des logiciels de CAO, connaissance des matériaux de construction. Exemples d'établissements : ENSAP (Bordeaux, Paris, Lille), École Spéciale d'Architecture (Paris).


In [43]:
# Test du chatbot
question_utilisateur = "Qu'est ce que le métier d'orthophoniste?"
print("Edu Pilot:", trouver_reponse(question_utilisateur))

Edu Pilot: L'orthophoniste évalue et rééduque les troubles de la parole, du langage, de la voix et de la déglutition. Il intervient auprès des enfants (retard de langage, dyslexie), des adultes (troubles neurologiques) et des personnes âgées (troubles de la déglutition). Compétences : patience, pédagogie, connaissances en linguistique, psychologie et anatomie. Exemples d'établissements : Universités avec un département d'orthophonie (Paris, Lille, Lyon, Marseille). L'admission se fait par concours très sélectif après le bac ou via une année de préparation.


## 6. 💾 Sauvegarde du Modèle Chatbot

Ce code permet de **sauvegarder notre modèle chatbot** afin de pouvoir le réutiliser sans avoir à recalculer la vectorisation TF-IDF à chaque exécution.

### 🔹 Fonctionnement :
1. **Utilisation de `pickle`**, une bibliothèque permettant de sérialiser (sauvegarder) des objets Python.
2. **Ouverture d'un fichier `chatbot_model.pkl` en mode écriture binaire (`wb`)**.
3. **Sauvegarde des éléments clés du modèle** :
   - `vectorizer` : le transformateur TF-IDF entraîné.
   - `tfidf_matrix` : la matrice des questions vectorisées.
   - `df` : le dataset utilisé.

✅ Une fois ce fichier créé, nous pourrons **charger directement ces objets** dans un autre script sans avoir à refaire la vectorisation des questions.

💡 **Essayez de modifier la base de données et de sauvegarder un nouveau modèle, puis rechargez-le pour voir les effets des changements !**


In [69]:
import pickle
with open("chatbot_model.pkl", "wb") as f:
    pickle.dump((vectorizer, tfidf_matrix, df), f)

print("✅ Modèle chatbot enregistré dans 'chatbot_model.pkl'")

✅ Modèle chatbot enregistré dans 'chatbot_model.pkl'


### 7. Initialtion au Fine_tuning :

Afin d'optimiser le modèle, utilisez des modèles pré-entrainé du type Mistral de hugging face, spécialiser à la langue française, ou encore GPT.

Lorsque nous lancerons nos données enrichies sur ce type de modèle LLM, nous appelons cela du Fine_tuning.

In [1]:
# Install PyTorch
%pip install torch

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/DialoGPT-small"  # Mets le nom du modèle que tu veux utiliser nous utilison DialoGPT-small, vous pouvez tester Mistral ou un autre LLM

# Téléchargement et sauvegarde locale
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sauvegarde en local
model.save_pretrained("chatbot_model")
tokenizer.save_pretrained("chatbot_model")

print("✅ Modèle et tokenizer sauvegardés en local dans chatbot_model/")

  Using cached torch-2.2.2-cp310-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (4.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp310-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached filelock-3.17.0-py3-none-any.whl (16 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_10_9_universal2.whl (14 kB)
Using cached mpmath-1.3.0-py3-none-

ModuleNotFoundError: No module named 'transformers'

Maintenant que vous avez vu comment nous utilisons des modèles LLM sur des bases de données.

A vous d'enrichir votre base de données avant de lancer le Fine_tuning sur votre dataset lors du prochain sprint.

Votre objectif :
*   Mettre en place une stratégie d'enrichissement de la base de données
*   Exécuter la statégie pour avoir un dataset enrichi au prochain samedi.



